In [43]:
import pandas as pd
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

In [42]:
load_dotenv()

True

In [14]:
df = pd.read_csv("epi_r.csv")
df.head()


,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
nan_summary = df[["title", "rating", "calories", "protein", "fat", "sodium"]].isna().sum()

# Count missing fields
nan_rows = df[["title", "rating", "calories", "protein", "fat", "sodium"]].isna().any(axis=1).sum()

nan_summary, nan_rows


(title          0
 rating         0
 calories    4117
 protein     4162
 fat         4183
 sodium      4119
 dtype: int64,
 np.int64(4188))

In [16]:
#drop recipes with missing nutrition
df_clean = df.dropna(subset=["calories", "protein", "fat", "sodium"])

#recheck
nan_summary = df_clean[["title", "rating", "calories", "protein", "fat", "sodium"]].isna().sum()
nan_rows = df_clean[["title", "rating", "calories", "protein", "fat", "sodium"]].isna().any(axis=1).sum()
nan_summary, nan_rows

(title       0
 rating      0
 calories    0
 protein     0
 fat         0
 sodium      0
 dtype: int64,
 np.int64(0))

In [53]:
#columns for ChatGPT cleaning
cols = df_clean.columns.to_list()
columns_str = ", ".join(cols)

#prompt
prompt = f"""
From the following list of column names, return only the ones that are food ingredients. Do not modify the ingredients spelling, grouping, etc at all, returning the ingredients written exactly as they were presented to you. 
Exclude anything that refers to metadata like cuisine, course, method, rating, or tags like vegan, kosher, etc.

Columns: {columns_str}

Only return a Python list of the ingredient columns.
"""

In [54]:
#call ChatGPT
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    )


response = client.chat.completions.create(
    model = "qwen/qwen-2.5-72b-instruct:free",
    messages=[
        {"role": "system", "content": "You are a helpful culinary assistant that extracts ingredient-related column names from data."},
        {"role": "user", "content": prompt}
    ], 

)
print(response.choices[0].message.content)

```python
['almond', 'amaretto', 'anchovy', 'anise', 'apple', 'apple juice', 'apricot', 'artichoke', 'arugula', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'basil', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry', 'biscuit', 'blackberry', 'blue cheese', 'blueberry', 'bok choy', 'bordeaux', 'bread', 'breadcrumbs', 'brie', 'brisket', 'broccoli', 'broccoli rabe', 'brown rice', 'brussel sprout', 'butter', 'buttermilk', 'butternut squash', 'cabbage', 'candy', 'cantaloupe', 'capers', 'caraway', 'cardamom', 'carrot', 'cashew', 'casserole/gratin', 'cauliflower', 'caviar', 'celery', 'champagne', 'chard', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea', 'chile', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro', 'cinnamon', 'clam', 'clove', 'cocktail', 'coconut', 'cod', 'coffee', 'cognac/armagnac', 'collard greens', 'condiment', 'condiment/spread', 'corn', 'cornmeal', 'cottage cheese', 'couscous', 'crab', 'cran

In [55]:
import ast

def parse_ingredient_list(raw_response):
    cleaned = raw_response.strip().removeprefix("```python").removesuffix("```").strip()
    return ast.literal_eval(cleaned)


In [59]:
#cleaned ingredient list
ingredient_list = parse_ingredient_list(response.choices[0].message.content)
print(ingredient_list)

['almond', 'amaretto', 'anchovy', 'anise', 'apple', 'apple juice', 'apricot', 'artichoke', 'arugula', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'basil', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry', 'biscuit', 'blackberry', 'blue cheese', 'blueberry', 'bok choy', 'bordeaux', 'bread', 'breadcrumbs', 'brie', 'brisket', 'broccoli', 'broccoli rabe', 'brown rice', 'brussel sprout', 'butter', 'buttermilk', 'butternut squash', 'cabbage', 'candy', 'cantaloupe', 'capers', 'caraway', 'cardamom', 'carrot', 'cashew', 'casserole/gratin', 'cauliflower', 'caviar', 'celery', 'champagne', 'chard', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea', 'chile', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro', 'cinnamon', 'clam', 'clove', 'cocktail', 'coconut', 'cod', 'coffee', 'cognac/armagnac', 'collard greens', 'condiment', 'condiment/spread', 'corn', 'cornmeal', 'cottage cheese', 'couscous', 'crab', 'cranberry', 'c

In [61]:
# Convert to lowercase for safe matching
ingredient_list = [i.lower() for i in ingredient_list]

# Clean and lowercase your DataFrame columns
df.columns = [col.lower() for col in df.columns]

# Keep only columns that match the ingredient list
ingredient_columns = [col for col in df.columns if col in ingredient_list]

# Filter the DataFrame
ingredients_df = df[ingredient_columns]

In [63]:
non_ingredients = [col for col in df.columns if col not in ingredient_columns]
print("Dropped columns:", non_ingredients)

Dropped columns: ['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '#cakeweek', '#wasteless', '22-minute meals', '3-ingredient recipes', '30 days of groceries', 'advance prep required', 'alabama', 'alaska', 'alcoholic', 'anniversary', 'anthony bourdain', 'aperitif', 'appetizer', 'arizona', 'asian pear', 'aspen', 'atlanta', 'australia', 'back to school', 'backyard bbq', 'bake', 'bass', 'bastille day', 'beverly hills', 'birthday', 'bitters', 'blender', 'boil', 'bon appétit', 'bon app��tit', 'boston', 'bourbon', 'braise', 'bran', 'brandy', 'breakfast', 'brine', 'broil', 'brooklyn', 'brownie', 'brunch', 'buffalo', 'buffet', 'bulgaria', 'bulgur', 'burrito', 'butterscotch/caramel', 'cake', 'california', 'calvados', 'cambridge', 'campari', 'camping', 'canada', 'candy thermometer', 'chambord', 'chartreuse', 'chicago', 'chill', 'christmas', 'christmas eve', 'cinco de mayo', 'citrus', 'cobbler/crumble', 'cocktail party', 'coffee grinder', 'colorado', 'columbus', 'connecticut', 'cook li

In [62]:
#select main columns
core_columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium']

#ingredient columns
ingredient_columns = ingredients_df

recipes_json = []

for _, row in df_clean.iterrows():
    ingredients = [col for col in ingredient_columns if row[col] == 1.0]
    
    if not ingredients:
        continue

    recipe = {
        "title": row['title'] ,
        "rating": row["rating"],
        "calories": row['calories'], 
        "macros":{
            "protein": row['protein'], 
            "fat": row['fat'], 
            "sodium": row['sodium']
        },
        "ingredients": ingredients
    }

    recipes_json.append(recipe)


In [64]:
output_path = "recipes.json"

with open(output_path, "w") as f:
    json.dump(recipes_json, f, indent = 2)
output_path

'recipes.json'